In [1]:
!pip install tensorflow

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


In [7]:
# Load the dataset
import pandas as pd
data = pd.read_csv("DailyDelhiClimateTest.csv")

# Use only the meantemp for prediction
values = data['meantemp'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values)

# Prepare the data for LSTM
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 3  # You can adjust the time step
X, y = create_dataset(scaled_values, time_step)

# Reshape input to be [samples, time steps, features] which is required for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, y_train, batch_size=1, epochs=50)


Epoch 1/50


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1087  
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - loss: 0.0107
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - loss: 0.0107
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - loss: 0.0105  
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - loss: 0.0118  
Epoch 6/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 917us/step - loss: 0.0111
Epoch 7/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - loss: 0.0107
Epoch 8/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - loss: 0.0091  
Epoch 9/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - loss: 0.0086
Epoch 10/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - loss: 0.0083
Epoch 11/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0079   
Epoch 12/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0090
Epoch 13/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0094    
Epoch 14/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - loss: 0.0091
Epoch 15/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 920us/step - loss: 

In [10]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions to get actual values
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

y_train = scaler.inverse_transform([y_train])
y_test = scaler.inverse_transform([y_test])


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [11]:
new_data = scaled_values[-time_step:].reshape(1, time_step, 1)
new_prediction = model.predict(new_data)
new_prediction = scaler.inverse_transform(new_prediction)

print(f'New Prediction: {new_prediction[0][0]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
New Prediction: 31.76100730895996


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


In [2]:
import pandas as pd 

In [3]:
# Load the dataset
data = pd.read_csv("DailyDelhiClimateTest.csv")

# Use only the meantemp for prediction
values = data['meantemp'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values)

# Prepare the data for LSTM
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 3  # You can adjust the time step
X, y = create_dataset(scaled_values, time_step)

# Convert data to PyTorch tensors
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=50, output_size=1):
        super(LSTM, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),
                            torch.zeros(1, 1, self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
